
# Title: Classifying the Stage of Heart Disease of Patient's from Cleveland based on Age, Resting Blood Pressure, Cholesterol, and Max Heart Rate.

## Group 30: Toby Lau, Neil Lin, Tanish Gupta, Kristy Fielding

### Background Information & Introduction

Heart disease is one of the leading causes of death across the United States of America that is caused by a wide variety of factors. Common heart diseases include coronary heart disease, arrhythmias, strokes, etc. Throughout the 21st century, medical professionals have deduced how certain factors may give leeway to one having a certain type of heart disease and the extent of its danger to the human body (Centers for Disease Control and Prevention, 2023). In this project, we will try to predict the severity of a patient's heart disease given the relevant predictors. The dataset being used is a collection of patients from Cleveland, Ohio, all of whom were suspected to have heart disease. The dataset contains 14 attributes that could contribute to the severity of one’s heart disease. Of the 14, we hope to train a model that is able to classify these stages given some attributes.

**Research Question: How can age, resting blood pressure, cholesterol, and max heart rate determine the stage of heart disease of patients from Cleveland?**

## Methods
To obtain a high quality model trained on the data used, we will have to do some data preprocessing and model tuning.

### Data reading and wrangling
1. Import the required libraries and load the dataset from the web
2. Assign column names and types to the dataset
3. Wrangle the dataset into a tidy format.
4. Split the data into training and testing sets
5. Summarize and visualize data

### Hyperparameter Tuning
1. Set a seed.
2. Create a recipe that defines the "stage" column as our response variable and everything else as its predictors.
3. Scale, center, and upscale the data using the SMOTE algorithm.
4. Create a $k$-nn model  with the neighbors = tune() parameter.
5. Create a tibble with a single neighbors column containing numbers from 1 to ???.
6. Perform Cross-validation ...
7. Plot the accuracy of the model against the number of neighbors to determine the $k$ value with the highest accuracy.
8. Using the $k$ value that obtained the highest accuracy, create a new $k$-nn model

### Evaluating
1. Test the model using our testing data set.
2. Check for over/underfitting.
3. Create a confusion matrix
4. Assess the model's performance using tidymodels.


## Discussion

## References